# TikTok

In [1]:
# pip install PyTikTokAPI
from TikTokAPI import TikTokAPI
import pandas as pd
import numpy as np
import math
import requests
import nest_asyncio
nest_asyncio.apply()
import pyppeteer
import pyppeteer_stealth
import time

In [2]:
# data_from_excel = pd.read_excel("Chốt số liệu Thanh Toán cho INFLU.xlsx", sheet_name="View", skiprows = 2, index_col= None)
sheet_id = '1a6oSAow7ZprMsGbbOv-bXEPJHo3y8nmPX5wimpyq1VQ'
sheet_name = 'Python'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}' 
data_from_excel = pd.read_csv(url)

In [ ]:
https://docs.google.com/spreadsheets/d/1a6oSAow7ZprMsGbbOv-bXEPJHo3y8nmPX5wimpyq1VQ/

In [3]:
data_from_excel.reset_index(inplace=True)
data_from_excel.rename(columns={"index": "STT"}, inplace=True)

In [4]:
data = data_from_excel
data.count()

STT             235
Mã INF          235
Kênh hợp tác    235
Link Video      235
dtype: int64

In [5]:
# Preprocessing data
data.dropna(subset=["Link Video"], inplace=True)
Tiktok_data = data[data['Link Video'].str.startswith('https://www.tiktok.com/')]

# get rid of unwanted new line and spacing
Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace('\n', '', regex=True)
Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace(' ', '', regex=True)

# get rid of '?...'
Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.split("?", expand=True)[0]

# Some links are from tiktoke main page
Tiktok_data = Tiktok_data[Tiktok_data['Link Video'].str.contains("/video/")]

<ipython-input-5-ccbfe1deb0ea>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace('\n', '', regex=True)
<ipython-input-5-ccbfe1deb0ea>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace(' ', '', regex=True)
<ipython-input-5-ccbfe1deb0ea>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [6]:
Tiktok_data.head()

,STT,Mã INF,Kênh hợp tác,Link Video
2,2,INFNHI376,TikTok,https://www.tiktok.com/@nhyyng/video/698771396...
3,3,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69862...
4,4,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69872...
5,5,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69872...
6,6,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69875...


In [7]:
cookie = {
  "s_v_web_id": "verify_kse1hozq_sL9KtG3W_wm1J_43ZU_8zFB_P0AlRDxsUSIO",
  "tt_webid": "6989095610934822402"
}

api = TikTokAPI(cookie=cookie)

In [8]:
import datetime
begin_time = datetime.datetime.now()

dict_list = [] # Store all the info
viewcount_list = [] # Store view counts

for link in Tiktok_data['Link Video'].values:
    video_id = int(link[-19:]) # Take the Id part

    try:
        retval = api.getVideoById(video_id)
        dict_list.append(retval)
        
        view = retval['itemInfo']['itemStruct']['stats']['playCount']
        
        item = {    
                    "Link Video": link,
                    "Số View": view,
        }
        
        viewcount_list.append(item)
        
    except:
        print("Fail!" + link)
    
df = pd.DataFrame(viewcount_list)
df = Tiktok_data.merge(df, on="Link Video", how="left")
df.to_pickle("./tiktok.pkl")

print(datetime.datetime.now() - begin_time)

Fail!https://www.tiktok.com/@phun.anh/video/6987227337142193435
Fail!https://www.tiktok.com/@behyblack/video/6985394076241661210
Fail!https://www.tiktok.com/@bthuy124/video/6985715429998185754
Fail!https://www.tiktok.com/@thinhnguyenart_official/video/6993871827055447298
Fail!https://www.tiktok.com/@anhtiendola/video/6993918759371066650
0:03:14.881616


In [9]:
Tiktok_datetime = datetime.datetime.now()
print(Tiktok_datetime)

2021-08-17 10:04:47.999661


In [12]:
# Function to print pretty dictionary
'''
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            pretty(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))
            
pretty(dict_list[0])
'''      

In [15]:
pd.read_pickle("Tiktok.pkl").head()

,STT,Mã INF,Kênh hợp tác,Link Video,Số View
0,2,INFNHI376,TikTok,https://www.tiktok.com/@nhyyng/video/698771396...,119800.0
1,3,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69862...,42100.0
2,4,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69872...,63300.0
3,5,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69872...,101100.0
4,6,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69875...,14800.0


# Instagram

In [11]:
# Importing packages
from selenium import webdriver
# Give access to Enter & ESC keys
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import time
import json

In [12]:
# Facebook Selenium Webscraping
#insta_data = data[data['Link Video'].str.startswith('https://www.instagram.com/')]
insta_data = data[data['Link Video'].str.contains('instagram')]

In [16]:
insta_data

,STT,Mã INF,Kênh hợp tác,Link Video
76,76,INFTOAN565,Instagram,https://www.instagram.com/reel/CRoQb_QDvXz/
77,77,INFTOAN565,Instagram,https://www.instagram.com/reel/CRydlrjDHH1/
78,78,INFTOAN565,Instagram,https://www.instagram.com/reel/CR1OPZPDuaP/
142,142,INFHAI912,Instagram,https://www.instagram.com/p/CRrBP9HFtOj/
194,194,INFDAI376,Instagram,https://www.instagram.com/p/CSRS1MgJjwA/?utm_m...


In [17]:
# Bypass notification
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

driver = webdriver.Chrome(chrome_options=chrome_options)

# Access google chrome and open insta login
driver.get("https://www.instagram.com/accounts/login/")

def click_fb_log():
    driver.find_element_by_class_name("KPnG0").click() #using Selenium click button method
def log_in():
    driver.find_element_by_name("email").send_keys("<Type in email>")
    time.sleep(2)
    driver.find_element_by_name("pass").send_keys("<Type in pass>")
    driver.find_element_by_id('loginbutton').click()

<ipython-input-17-84a1d52b201d>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


In [18]:
insta_list = []

for link in insta_data["Link Video"].values:
    base_url = link
    url_json = (base_url + '?__a=1')
    driver.get(url_json)
    
    data_j = driver.find_element_by_tag_name('body').text
    try:
        data_json = json.loads(data_j)

        view = data_json["graphql"]['shortcode_media']['video_view_count']

        item = {
                    "Link Video": link,
                    "Số View": view,
        }

        insta_list.append(item)
        time.sleep(.5)
    except:
        print("Cannot get insta info!")
    
df = pd.DataFrame(insta_list)
df = insta_data.merge(df, on="Link Video", how="left")
df.to_pickle("insta.pkl")

Cannot get insta info!


In [19]:
insta_datetime = datetime.datetime.now()
print(insta_datetime)

2021-08-17 10:06:20.245711


In [20]:
pd.read_pickle("insta.pkl").head()

,STT,Mã INF,Kênh hợp tác,Link Video,Số View
0,76,INFTOAN565,Instagram,https://www.instagram.com/reel/CRoQb_QDvXz/,405303.0
1,77,INFTOAN565,Instagram,https://www.instagram.com/reel/CRydlrjDHH1/,1492215.0
2,78,INFTOAN565,Instagram,https://www.instagram.com/reel/CR1OPZPDuaP/,568080.0
3,142,INFHAI912,Instagram,https://www.instagram.com/p/CRrBP9HFtOj/,1484.0
4,194,INFDAI376,Instagram,https://www.instagram.com/p/CSRS1MgJjwA/?utm_m...,NaN


# Facebook

In [21]:
fb_data = data[data['Link Video'].str.startswith('https://www.facebook.com/')]
fb_data = fb_data[fb_data['Link Video'].str.contains("/videos/")]

In [22]:
fb_data["Link Video"].values

array(['https://www.facebook.com/nguyen.huyenthu.716/videos/2920449061508309',
       'https://www.facebook.com/100006045494111/videos/781616739195419',
       'https://www.facebook.com/tienanh.vu.334/videos/358632378979133',
       'https://www.facebook.com/moclanla/videos/810057876567671',
       'https://www.facebook.com/100034376687766/videos/1011431982958642',
       'https://www.facebook.com/100028527371304/videos/557859528559826',
       'https://www.facebook.com/Changchi.1604/videos/165010988924001',
       'https://www.facebook.com/thanh.duong.5220/videos/133547462228191',
       'https://www.facebook.com/thanhhuyen.bgvn/videos/338390987868753',
       'https://www.facebook.com/dangducdai69/videos/1050642239009275',
       'https://www.facebook.com/chinchinnnnnn/videos/852336042329106/',
       'https://www.facebook.com/hermymeow/videos/359543159133729',
       'https://www.facebook.com/loanvu1312/videos/1131153590627327',
       'https://www.facebook.com/ky.ky.5209/videos/408

In [23]:
driver = webdriver.Chrome(chrome_options=chrome_options)

# Access google chrome and open insta login
driver.get("https://www.facebook.com/")

<ipython-input-23-fc674300ede4>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


In [24]:
fb_list = []

for link in fb_data["Link Video"].values:
 
    driver.get(link)
    try:
        view_count = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div[2]/div/div/div[1]/div/div/div[2]/div[1]/div/div[2]/div[2]/span/span")
        view = view_count.text[:-10].replace(",","") + "000"

        item = {    
                    "Link Video": link,
                    "Số View": view,
        }

        fb_list.append(item)
    except:
        print(link)
        
df = pd.DataFrame(fb_list)
df = fb_data.merge(df, on="Link Video", how="left")
df.to_pickle("fb.pkl")

fb_datetime = datetime.datetime.now()
print(fb_datetime)

https://www.facebook.com/thanhhuyen.bgvn/videos/338390987868753
https://www.facebook.com/dangducdai69/videos/1050642239009275
https://www.facebook.com/01633166673h/videos/1033006097103767
2021-08-17 10:07:42.833227


In [25]:
tiktok = pd.read_pickle("tiktok.pkl")
insta = pd.read_pickle("insta.pkl")
fb = pd.read_pickle("fb.pkl")

In [26]:
merge = pd.concat([tiktok, insta, fb])

In [44]:
merge1 = merge[["STT", "Số View"]]
merge1.dropna(subset=["Số View"], inplace=True)
merge1["Số View"] = merge1["Số View"].apply(lambda x: math.floor(int(x)/1000))
export_df = data_from_excel.merge(merge1, on="STT", how="left")

<ipython-input-44-50a188929954>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge1.dropna(subset=["Số View"], inplace=True)
<ipython-input-44-50a188929954>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge1["Số View"] = merge1["Số View"].apply(lambda x: math.floor(int(x)/1000))


In [45]:
time_df = pd.DataFrame({"Ngày giờ" :["Tiktok : " + str(Tiktok_datetime), "Insta : " + str(insta_datetime), "Fb : " + str(fb_datetime)]})
export_df = pd.concat([export_df, time_df], axis=1)

In [46]:
export_df

,STT,Mã INF,Kênh hợp tác,Link Video,Số View,Ngày giờ
0,0,INFHUYEN641,Facebook,https://www.facebook.com/nguyen.huyenthu.716/v...,10.0,Tiktok : 2021-08-17 10:04:47.999661
1,1,INFHUYEN641,Facebook,https://www.facebook.com/100008065904636/posts...,NaN,Insta : 2021-08-17 10:06:20.245711
2,2,INFNHI376,TikTok,https://www.tiktok.com/@nhyyng/video/698771396...,119.0,Fb : 2021-08-17 10:07:42.833227
3,3,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69862...,42.0,NaN
4,4,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69872...,63.0,NaN
...,...,...,...,...,...,...
232,230,INFHAN236,TikTok,https://www.tiktok.com/@chutieuhan10012001/vid...,9500.0,NaN
233,231,INFCUONG506,TikTok,https://www.tiktok.com/@cuonghancouple/video/6...,8300.0,NaN
234,232,INFCUONG506,TikTok,https://www.tiktok.com/@cuonghancouple/video/6...,620.0,NaN
235,233,INFCUONG506,TikTok,https://www.tiktok.com/@cuonghancouple/video/6...,985.0,NaN


In [28]:
# Export file to excel -> push it to gsheet
# export_df.to_excel("export_data.xlsx", index=False, sheet_name="View")

# Gsheet Export

In [50]:
import gspread
from gspread_dataframe import set_with_dataframe

# ACCES GOOGLE SHEET
gc = gspread.service_account(filename=r'GoogleAPICredentials.json') # <--Key
sh = gc.open_by_key(sheet_id) # <-- sheet id
worksheet = sh.worksheet('Python') #->gg sheet name 

# APPEND DATA TO SHEET
your_dataframe = export_df
set_with_dataframe(worksheet, your_dataframe) #-> THIS EXPORTS YOUR DATAFRAME TO THE GOOGLE SHEET